# basic idea
- 기존 ML 기반 클러스터링은 잘 작동하지 않았음
- 텍스트가 주제 분류될 때 복잡한 과정을 거치기 때문 ex) KT wiz와 삼성 라이온즈의 맞대결 승자는? &rarr; 스포츠; 하지만 KT와 삼성이라는 단어 때문에 경제로 분류될 가능성 있음.
- 복잡한 로직의 처리를 LLM으로 하면 어떨까? &rarr; LLM 사용 시 출력 컨트롤이 관건
    1. 출력이 영어로 되는 경우 - 프롬프트에서 한국어 명시
    2. 출력이 균일하게 되지 않는 경우 - seed와 temperature 관리
    3. 출력에 노이즈가 생기는 경우 - 노이즈 양에 따라 허용 or 무시

In [16]:
import torch
import re
import pandas as pd
import os

from filter import SpecialCharFilter

from langchain_ollama import ChatOllama
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from sklearn.cluster import KMeans
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

In [2]:
df: pd.DataFrame = pd.read_csv('data/train.csv')

special_char_filter = SpecialCharFilter()

noise_df = special_char_filter.filter_noise(df)
clean_df = df[~df.index.isin(noise_df.index)]
print(len(noise_df), len(clean_df))
noise_df.head()

1595 1205


,ID,text,target,special_char_count,special_char_ratio
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,6,0.187500
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,7,0.259259
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,5,0.227273
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,5,0.178571
6,ynat-v1_train_00006,프로야구~롯TKIAs광주 경기 y천취소,1,3,0.142857


In [3]:
prompt = ChatPromptTemplate.from_messages(
    messages=[
        (
            "system",
            """
            You are a helpful assistant that categories news article titles to propre sections. 
            only say in a short korean word.
            """,
        ),
        (
            "human",
            "{input}",
        )
    ]
)

llm = ChatOllama(
    model="gemma2:27b",
    seed=42,
    temperature=0
)

chain = prompt | llm

In [4]:
cluster_path = "data/clustered.csv"

if not os.path.exists(cluster_path):
    for idx, row in clean_df.iterrows():
        input_text = row['text']
        ai_msg = chain.invoke({"input": input_text})
        predicted_label = ai_msg.content.strip()  # 결과 문자열에서 공백 제거
        clean_df.loc[idx, 'predict_label'] = re.sub(r'[^가-힣a-zA-Z\s]', '', predicted_label)
        print(f"Index: {idx}, Input: {input_text}, Predicted Label: {predicted_label}")
else:
    clean_df = pd.read_csv(cluster_path)

위 실행 결과를 보면, 다양한 label로 분류되는 걸 알 수 있다. 따라서 label 텍스트 자체를 임베딩해 클러스터링을 시도한다.

# 실험 1: BERT embedding + K-Means

In [5]:
clean_df.to_csv('data/clustered.csv')

In [6]:
label_counts = clean_df['predict_label'].value_counts()
label_counts.head(10)

predict_label
경제       189
정치       158
스포츠       71
기술        62
사회        61
스포츠       52
국제        44
날씨        35
과학        33
국제 정치     31
Name: count, dtype: int64

klue/bert-base 모델을 활용해 토큰화 및 임베딩 후 k-means 알고리즘으로 클러스터링을 한다.

In [7]:
model_name = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [8]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=32, padding="max_length")
    outputs = model(**inputs)
    # [CLS] 토큰의 출력만 사용하여 임베딩 생성
    embedding = outputs.last_hidden_state[:, 0, :].squeeze().detach().numpy()
    return embedding

In [9]:
embeddings = clean_df['predict_label'].astype(str).apply(get_embedding).tolist()
embeddings = torch.tensor(embeddings)  # 리스트를 텐서로 변환

# KMeans 클러스터링 (7개의 클러스터로 설정)
kmeans = KMeans(n_clusters=7, random_state=42)
clean_df['label_cluster'] = kmeans.fit_predict(embeddings)

# 클러스터링 결과 확인
print(clean_df[['predict_label', 'label_cluster']].head())

/var/folders/f9/7w3cw7_s6c7849zf1svk37000000gn/T/ipykernel_94362/601033594.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:264.)
  embeddings = torch.tensor(embeddings)  # 리스트를 텐서로 변환
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  predict_label  label_cluster
0            경제              4
1            사회              2
2           농구               3
3            사회              2
4          전자제품              6


In [10]:
cluster_counts = clean_df['label_cluster'].value_counts()
print("\nCluster distribution:")
print(cluster_counts)


Cluster distribution:
label_cluster
4    329
0    277
1    265
3    155
2    106
6     54
5     19
Name: count, dtype: int64


In [11]:
cross_tab = pd.crosstab(clean_df['label_cluster'], clean_df['target'])
cross_tab

target,0,1,2,3,4,5,6
label_cluster,,,,,,,
0,36,40,36,40,36,49,40
1,41,39,48,29,37,31,40
2,19,18,11,10,17,19,12
3,23,29,11,27,27,20,18
4,43,45,43,42,53,55,48
5,2,5,1,8,1,1,1
6,6,7,10,3,8,14,6


클러스터링 실험 결과 사전에 분석한 라벨 분포와 다르다. 사전 분석 시 라벨이 골고루 분포되어 있었다.

# 실험 2: LLM 라벨 예측 결과 분류 - 대화 정보 기억

LLM의 출력 결과를 7가지로 추리기 위해서 어떤 기능을 사용해야 할까?
LLM agent에서 버퍼 메모리 기능 사용해보기

In [12]:
store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chain = RunnableWithMessageHistory(llm, get_session_history)

In [13]:
def extract_categories(titles, session_id):
    prompt = (
        "다음은 뉴스 기사 제목들의 목록입니다:\n\n"
        + "\n".join(titles)
        + "\n\n이 제목들을 7개의 대표적인 카테고리로 분류하여, 각 카테고리의 이름만 한 단어로 출력하세요."
    )
    response = chain.invoke(
        prompt,
        config={"configurable": {"session_id": session_id}},
    )
    return response.content.strip()

def classify_title(title, categories, session_id):
    prompt = (
        f"다음은 뉴스 기사 제목입니다:\n\n"
        f"제목: {title}\n\n"
        f"아래의 카테고리 중 하나로 분류하고, 해당 카테고리의 이름만 한 단어로 출력하세요:\n{', '.join(categories)}"
    )
    response = chain.invoke(
        prompt,
        config={"configurable": {"session_id": session_id}},
    )
    return response.content.strip()

In [ ]:
# 예시 기사 제목 리스트
titles = clean_df['text'].to_list()

# 세션 ID 설정
session_id = "1"

# 1. 카테고리 추출
categories = extract_categories(titles, session_id)
print("추출된 카테고리:\n", categories)

# 2. 각 제목에 대한 분류
classified_categories = []
for title in tqdm(titles):
    category = classify_title(title, categories, session_id)
    classified_categories.append(categories)
    print(f"제목: {title}\n분류된 카테고리: {category}\n")
    
clean_df['label_cluster'] = classified_categories
clean_df.to_csv('data/clustered_7_labels.csv')

추출된 카테고리:
 1. 정치
2. 경제
3. 스포츠
4. 사회
5. 과학
6. 문화
7. 국제


  0%|          | 1/1205 [00:01<31:58,  1.59s/it]

제목: 갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩
분류된 카테고리: 경제



  0%|          | 2/1205 [00:02<29:38,  1.48s/it]

제목: 美성인 6명 중 1명꼴 배우자 연인 빚 떠안은 적 있다
분류된 카테고리: 사회



...

제목: 안보리 대북결의안 2270호 이행보고서 제출한 나라 70개 육박
분류된 카테고리: 정치



  1%|▏         | 16/1205 [01:40<4:33:58, 13.83s/it]

제목: 게시판 KBS 코로나가 바꾼 일상 대국민 영상 공모
분류된 카테고리: 문화



  1%|▏         | 16/1205 [01:56<2:23:53,  7.26s/it]


KeyboardInterrupt: 